In [1]:
!pip install spacy nltk

  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl (29 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached wasabi-1.1.2-py3-none-any.whl.metadata (28 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached weasel-0.3.4-py3-none-any.whl.metadata (4.7 kB)
  Using cached typer-0.9.0-py3-none-any.whl (45 kB)
  Using cached pydantic-2.5.2-py3-none-any.whl.metadata (65 kB)
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
  Using cached annotated_types-0.6.0-py3-none-any.whl.metadata (12 kB)
  Using cached confection-0.1.4-py3-none-any.whl.metadata (19 kB)
  Using cached cloudpathlib-0.16.0-py3-none-any.whl.metadata (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 8.0 MB/s eta 0:00:0000:0100:01
Using cached catalogue-2.0.10-py3-none-any.whl (17 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 4.0 MB/s eta 0:00:0

In [2]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 7.3 MB/s eta 0:00:0000:0100:01


In [ ]:
import nltk
nltk.download()

In [9]:
print('Loading Classifiers...')
from Classifiers.Classifier import Classifier
from Classifiers.XGBoost import XGBoost
#from TextProc.Preprocessor import Preprocessor
import configs
import argparse
from datetime import datetime
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from copy import deepcopy
import os
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as stats
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import nevergrad as ng
from sklearn.metrics import accuracy_score
from transformers import BertTokenizer, BertModel
import torch








Loading Classifiers...


OSError: [E050] Can't find model 'en_core_web_md'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

    # Function to encode text data using BERT
def bert_encode(texts, tokenizer, model, max_len=512):
        input_ids = []
        attention_masks = []

        for text in texts:
            encoded = tokenizer.encode_plus(
                text,
                max_length=max_len,
                truncation=True,
                padding='max_length',
                add_special_tokens=True,
                return_attention_mask=True,
                return_tensors='pt',
            )
            input_ids.append(encoded['input_ids'])
            attention_masks.append(encoded['attention_mask'])

        input_ids = torch.cat(input_ids, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)

        with torch.no_grad():
            last_hidden_states = model(input_ids, attention_mask=attention_masks)

        # Extract the tokens' embeddings
        features = last_hidden_states[0][:,0,:].numpy()
        return features

In [7]:
class TrainEngine():
    def __init__(self, train, config, train_data_path, save_path):
        self.train = train
        self.config = config
        self.train_data_path = train_data_path
        self.save_model_path = save_path

        # intialize classifier and preprocessor
        self.model = self.choose_model(config['model_config'])
        #self.preprocessor = Preprocessor(config['preproc_config'])


    def load_data(self, data_path: str, train: bool = True) -> pd.DataFrame:
        """
        loads data from a csv file
        :param path: path to csv file
        :return: pandas dataframe
        """
        # load the dataset of texts one text per line
        pos_path = '{}/train_pos_preprocessed.txt'.format(data_path)
        neg_path = '{}/train_neg_preprocessed.txt'.format(data_path)
        test_path = '{}/test_data.txt'.format(data_path)

        if train:
            with open(pos_path, 'r') as f:
                train_pos = f.readlines()

            with open(neg_path.format(data_path), 'r') as f:
                train_neg = f.readlines()

            # create a dataframe with the text and label
            df = pd.DataFrame({'text': np.concatenate([train_pos, train_neg]),
                                'label': np.concatenate([np.ones(len(train_pos)), np.zeros(len(train_neg))])
                                })
        else:
            with open(test_path, 'r') as f:
                test = f.readlines()
            df = pd.DataFrame({'text': test})
            # split text on the first comma
            df['Id'] = df['text'].apply(lambda x: x.split(',', 1)[0])
            df['text'] = df['text'].apply(lambda x: x.split(',', 1)[1])
        
        print('size of training data:', df.shape)
        return df

    
    def save_metrics(self, metrics: dict, path: str) -> None:
        """
        saves metrics to a csv file
        :param metrics: metrics to be saved
        :param path: path to csv file
        """
        pd.DataFrame.from_dict(metrics, orient='index').to_csv(path)

    def save_predictions(self, predictions: list, path: str) -> None:
        """
        saves predictions to a csv file
        :param predictions: predictions to be saved
        :param path: path to csv file
        """
        pd.DataFrame(predictions).to_csv(path)

    def choose_model(self, model_config: dict) -> Classifier:
        """
        chooses a model based on the model_type
        :param model_type: type of model to be chosen
        :return: model
        """
        # if model_config['model_type'] == 'bert':
            # return BERT(model_config)
        if model_config['model_type'] == 'xgboost':
            return XGBoost(model_config)
        # elif model_config['model_type'] == 'xlnet':
        #     # add item to model_config
        #     mc = deepcopy(model_config)
        #     mc['output_dir'] = self.save_model_path
        #     mc['best_model_dir'] = self.save_model_path + '/best_model/'
        #     mc['cache_dir'] = self.save_model_path + '/cache_dir/'
        #     mc['tensorboard_dir'] = self.save_model_path + '/tensorboard_dir/'
        #     return XLNet(mc)
        else:
            raise ValueError('Model type not supported')
        
    def perform_random_search(self, X_train, y_train, param_dist, n_iter=10, cv=5, scoring='accuracy'):
        """
        Perform random search for hyperparameter tuning.
        :param X_train: Training features
        :param y_train: Training labels
        :param param_dist: Dictionary with parameters names as keys and distributions or lists of parameters to try.
        :param n_iter: Number of parameter settings that are sampled.
        :param cv: Number of folds in cross-validation.
        :param scoring: Strategy to evaluate the performance of the cross-validated model on the test set.
        :return: Fitted RandomizedSearchCV object
        """
        if not isinstance(self.model, XGBoost):
            raise ValueError("Random search is currently only implemented for XGBoost models")

        random_search = RandomizedSearchCV(self.model.xgb_classifier, param_distributions=param_dist, 
                                           n_iter=n_iter, cv=cv, scoring=scoring, verbose=1, random_state= 42)
        random_search.fit(X_train, y_train)
        return random_search

    def print_metrics(self, metrics: dict) -> None:
        """
        prints the metrics
        :param metrics: metrics to be printed
        """
        for key, value in metrics.items():
            print('{}: {}'.format(key, value))
            
    def create_xgboost_model(self, hyperparams):
        """
        Create a new instance of the XGBoost model with given hyperparameters.
        """
        config = self.config['model_config']
        config.update(hyperparams)
        return XGBoost(config)
            
            
    def objective_function(self, hyperparams):
        """
        Objective function for Nevergrad optimization.
        """
        # Create a new XGBoost model instance with the given hyperparameters
        model = self.create_xgboost_model(hyperparams)

        # Train the model
        model.train(self.X_train, self.y_train)

        # Validate the model
        predictions = model.predict(self.X_val)

        # Calculate and return the negative accuracy (since Nevergrad minimizes)
        accuracy = accuracy_score(self.y_val, predictions)
        return -accuracy
    
    def optimize_hyperparameters(self):
        """
        Use Nevergrad to optimize hyperparameters.
        """
        # Define the hyperparameter space
        parametrization = ng.p.Dict(
            learning_rate=ng.p.Scalar(lower=0.01, upper=0.2).set_mutation(sigma=0.01),
            max_depth=ng.p.Scalar(lower=3, upper=10).set_integer_casting(),
            n_estimators=ng.p.Scalar(lower=100, upper=1000).set_integer_casting(),
            subsample=ng.p.Scalar(lower=0.5, upper=1)
        )

        # Choose the optimizer
        optimizer = ng.optimizers.NGOpt(parametrization=parametrization, budget=100)

        # Run the optimization
        for _ in range(optimizer.budget):
            x = optimizer.ask()
            loss = self.objective_function(x.value)
            optimizer.tell(x, loss)

        # Get the best hyperparameters
        best_hyperparams = optimizer.provide_recommendation().value

        # Evaluate the objective function with the best hyperparameters
        best_score = self.objective_function(best_hyperparams)

        # Convert the score to a positive value, if necessary
        best_accuracy = -best_score if best_score < 0 else best_score

        return best_hyperparams, best_accuracy
    


    def run(self):
        print('Loading data...')
        X = self.load_data(self.train_data_path, train=self.train)
        print(X.head())
        if self.train:
            # load data
            y = X['label']
            # drop label column
            X.drop('label', axis=1, inplace=True)

            print('Preprocessing data...')
            # preprocess data
            #X = self.preprocessor.preprocess(X)
            
            print("Type of X:", type(X))

            
            text_column = X['text']

            # Now apply TF-IDF Vectorization
            vectorizer = TfidfVectorizer(max_features=5000)
            X_vectorized = vectorizer.fit_transform(text_column)
            
            
            
            #X_vectorized = bert_encode(text_column, tokenizer, model)
         
            
            # split data into train and validation
            X_train, X_val, y_train, y_val = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)
            # convert to list
            y_train = np.array(y_train.tolist())
            y_val = np.array(y_val.tolist())
            y_val = np.array([-1 if y == 0 else 1 for y in y_val])
            
            param_dist = {
                'max_depth': stats.randint(3, 10),
                'learning_rate': stats.uniform(0.01, 0.1),
                'subsample': stats.uniform(0.5, 0.5),
                'n_estimators':stats.randint(50, 200)
            }
            
            print(X_train.shape)
            print(X_val.shape)
            
            # Define the hyperparameter grid
            param_grid = {
                'max_depth': [3, 5, 7],
                'learning_rate': [0.1, 0.01, 0.001],
                'subsample': [0.5, 0.7, 1]
            }

            # print('Performing Random Search for Hyperparameter Tuning...')

            # random_search_result = self.perform_random_search(X_train, y_train, param_dist)
            
            # print("Best set of hyperparameters: ", random_search_result.best_params_)
            # print("Best score: ", random_search_result.best_score_)
            
            params = {
                'learning_rate': 0.06986584841970366,
                'max_depth': 9,
                'n_estimators': 171,
                'subsample': 0.5779972601681014
            }
            
            self.X_train, self.X_val, self.y_train, self.y_val = X_train, X_val, y_train, y_val


            #print('Optimizing hyperparameters...')
            #best_hyperparams, best_accuracy = self.optimize_hyperparameters()
            #print('Best hyperparameters:', best_hyperparams)
            #print('Best accuracy:', best_accuracy)

            # Set the best hyperparameters to the model
            #self.model.set_params(**best_hyperparams)
            

            print('Training...')
            # train
            self.model.train(X_train, y_train)
            # dtrain = xgb.DMatrix(X_train, label=y_train)
            # self.model = xgb.train(params, dtrain)

            print('Saving model...')
            # save model
            self.model.save(self.save_model_path)

            print('Testing...')
            # test
            self.model.validate(X_val, y_val)

            print('Metrics:')
            # get metrics
            metrics = self.model.get_metrics()
            self.print_metrics(metrics)
            
            
            
        else:
            # load data
            X = self.load_data(self.train_data_path, train=False)
            # preprocess data
            X['text'] = self.preprocessor.preprocess(X['text'].to_numpy())
            # convert to list
            X_val = X['text'].to_numpy()

            print('Predicting...')
            # predict
            X['label'] = self.model.predict(X_val)

            print('Saving predictions...')
            # save predictions
            self.save_predictions(X[['Id', 'Prediction']], self.save_model_path)

        print('Finished Running !')

In [8]:
print('Getting current date time...')
now = datetime.now()
current_time = now.strftime("%m-%d-%Y-%H:%M:%S")

print('Parsing arguments...')
# Instead of argparse, directly assign values to variables
train_config = 'xgboost_config_train'
config = configs.get_config(train_config)
train = True
train_data_path = 'Data/twitter-datasets'
save_model_path = f'Models/model_{current_time}'

print('Loading config...')
config = configs.get_config(train_config)

print('Initializing train engine...')
engine = TrainEngine(train, config, train_data_path, save_model_path)

print('Running training...')
engine.run()


Getting current date time...
Parsing arguments...


NameError: name 'configs' is not defined